<!--
Doc Writer email@nixdabei.de
v0.0.1, 2022-02-19
-->
[Home](../index.ipynb) /  Leistungstreiber L9110: Simple car
***
# Leistungstreiber L9110: Simple Car

## Klasse: Motor

In [51]:
%serialconnect #--port=COM3 # Windows with more than one COM-Port

import machine

class Motor :
    def __init__( self, 
            iPinA, 
            iPinB,
            iFrequency     = 1000,
            iDutyMinLeft   =    0,  # Duty for speed = 0:              motor turns left
            iDutyMaxLeft   = 1023,  # Duty for speed = fSpeedMaxLeft:  motor turns left
            iDutyMinRight  =    0,  # Duty for speed = 0:              motor turns right
            iDutyMaxRight  = 1023,  # Duty for speed = fSpeedMaxRight: motor turns right
            fSpeedMaxLeft  = -1.0,  # Minimal value for setSpeed() --> iDutyMaxLeft
            fSpeedMaxRight =  1.0   # Maximal value for setSpeed() --> iDutyMaxRight
        ) :
        self._iFrequency    = iFrequency
        self._fScaleLeft    =  (iDutyMaxLeft  - iDutyMinLeft )/fSpeedMaxLeft
        self._fScaleRight   =  (iDutyMaxRight - iDutyMinRight)/fSpeedMaxRight
        self._iDutyMinLeft  = iDutyMinLeft
        self._iDutyMinRight = iDutyMinRight
        
        self._fCurSpeed     = 0
        
        self._pwmA = machine.PWM( machine.Pin(iPinA), freq=self._iFrequency, duty=0)
        self._pwmB = machine.PWM( machine.Pin(iPinB), freq=self._iFrequency, duty=0)
        
        self._pwmDir   = self._pwmA
        self._pwmSpeed = self._pwmB
        
    def speedToDuty( self, fSpeed ) :
        if fSpeed == 0 :
            return 0
        elif fSpeed < 0 :
            return int(self._iDutyMinLeft  + fSpeed*self._fScaleLeft +0.5)
        else :
            return int(self._iDutyMinRight + fSpeed*self._fScaleRight+0.5)
            
    
    def setLeft( self ) :
        self._pwmDir   = self._pwmA
        self._pwmDir.duty( 0 )
        self._pwmSpeed = self._pwmB
    
    def setRight( self ) :
        self._pwmDir   = self._pwmB
        self._pwmDir.duty( 0 )
        self._pwmSpeed = self._pwmA
    
    def setSpeed( self, fSpeed ) :
        if fSpeed != self._fCurSpeed :
            iDuty = self.speedToDuty( fSpeed )
                
            if fSpeed > 0 and self._fCurSpeed <= 0 :
                self.setLeft()

            elif fSpeed < 0 and self._fCurSpeed >= 0 :
                self.setRight()
        
            self._pwmSpeed.duty( iDuty )
        
        self._fCurSpeed = fSpeed


Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


## Simple Car

In [52]:
from time import sleep_ms

DELAY_MILLIS = 20

I_STEPS     = 100
F_SPEED_MIN = -1.0
F_SPEED_MAX =  1.0


motorL = Motor( 12, 13,
    iFrequency     = 100,
    iDutyMinLeft   = 330,
    iDutyMaxLeft   = 1023,
    iDutyMinRight  = 330,
    iDutyMaxRight  = 1023,
    fSpeedMaxLeft  = -1.0,
    fSpeedMaxRight =  1.0 )

motorR = Motor( 18, 22,
    iFrequency     = 100,
    iDutyMinLeft   = 330,
    iDutyMaxLeft   = 1023,
    iDutyMinRight  = 330,
    iDutyMaxRight  = 1023,
    fSpeedMaxLeft  = -1.0,
    fSpeedMaxRight =  1.0 )


fSpeed    = F_SPEED_MIN
fStep     = (F_SPEED_MAX - F_SPEED_MIN)/I_STEPS


try:    
    for i in range( 100 ):
        while fSpeed <= F_SPEED_MAX:
            #motorL.setSpeed(  fSpeed )
            motorR.setSpeed( -fSpeed )
            sleep_ms(DELAY_MILLIS)
            fSpeed += fStep
        sleep_ms( 1000 )
        while fSpeed >= F_SPEED_MIN:
            motorL.setSpeed(  fSpeed )
            #motorR.setSpeed( -fSpeed )
            sleep_ms(DELAY_MILLIS)
            fSpeed-=fStep
            
except KeyboardInterrupt: pass
        
motorL.setSpeed( 0 )
motorR.setSpeed( 0 )
    
print( "Done" )        

...

*** Sending Ctrl-C

Done
